# [![](https://img.shields.io/badge/Musubi-Tuner-Blue)](https://github.com/kohya-ss/musubi-tuner) [![](https://img.shields.io/badge/Video-Hướng%20dẫn-ff0000)](https://youtu.be/CHUAkrPhoig) [![](https://img.shields.io/badge/Design-stablediffusion.vn-0075ff)](https://stablediffusion.vn) [![](https://img.shields.io/badge/Ver-26.0102-0075ff)](https://trainlora.vn) [![](https://img.shields.io/badge/All%20Tools-trainlora.vn-0075ff)](https://trainlora.vn) [![](https://img.shields.io/badge/SDVN-Library-green)](https://bit.ly/sdvn-lib) [![](https://img.shields.io/badge/Khoá%20học-All%20in%20one-red)](https://hungdiffusion.com/) [![](https://img.shields.io/badge/Group-Support-0075ff)](https://www.facebook.com/groups/stablediffusion.vn) [![](https://img.shields.io/discord/813085864355037235?color=blue&label=Discord&logo=Discord)](https://discord.gg/5SEtApPeyG)
---
`✨ Hỗ trợ Qwen 2511, Z-Image, Finetuning, Wan`

`✨ Thêm các option mở rộng`

#⚙️ I. Cài đặt

In [ ]:
# @title ###Install
import os
from google.colab import drive

drive.mount('/content/drive')
root_dir = "/content/"
data_dir = "/content/drive/MyDrive/SD-Data"
repo_dir = "/content/SDVN-musubi-tuner-colab"

if not os.path.exists("/content/musubi-tuner"):
  %cd /content
  !git clone https://github.com/StableDiffusionVN/SDVN-musubi-tuner-colab
  !git clone https://github.com/kohya-ss/musubi-tuner
  %cd /content/musubi-tuner
  !pip install -q -e .
  !pip install -q toml aria2 OpenAI google-genai

%cd /content
%run {repo_dir}/TrainScript.ipynb

# 🎚️ II. Train Image (Qwen, Kontext)

In [ ]:
#@title 📂 2. Kết nối - xử lý data

ImageFolder = "/content/drive/MyDrive/SD-Data/TrainData"  # @param {type:'string'}
# @markdown 🟡 `Thư mục control chỉ dùng cho train Kontext, Qwen-Edit`

# @markdown 🟡 `Đặt tên, vị trí thư mục như ImageFolder, đặt hậu tố _{số} (control_0.jpg) nếu muốn train nhiều control cho một image`
ControlFolder = ""  # @param {type:'string'}
DataClean = False #@param {type:"boolean"}
No_gen_caption = False #@param {type:"boolean"}
# @markdown 🟡 `OCR Prompt`

#@markdown [![](https://img.shields.io/badge/Caption-Tools-blue)](https://colab.research.google.com/github/StableDiffusionVN/SDVN-training-colab-flux/blob/main/SDVN_Caption_Tools.ipynb)

Caption_Image = 'Florence' # @param ['None', 'Florence', 'APIGemini | 2.5 Flash', ' APIGemini | 2.5 Pro', 'APIGemini | 2.5 Flash Lite',  'APIOpenAI | GPT 5', 'APIOpenAI | GPT 5-mini', 'APIOpenAI | GPT 5-nano']
Caption_Length = "Medium" # @param ["Short","Medium","Long"]
# @markdown 🟡 `API Caption`

# @markdown [![](https://img.shields.io/badge/Gemini-API-blue)](https://aistudio.google.com/app/apikey)
APIkey = "" # @param {type:'string'}
API_Prompt = "" # @param {type:'string'}
# @markdown 🟡 `Thêm caption tuỳ chọn`
Custom_Caption = "" # @param {type:'string'}
AddFolderName = False #@param {type:"boolean"}
Remove_Caption = False #@param {type:"boolean"}
Append = False #@param {type:"boolean"}

if No_gen_caption == True:
  Caption_Image = 'None'
  Caption_Video = 'None'
  Custom_Caption = ''
  AddFolderName = False

Cap_prompt = {
    'Short':['<CAPTION>',0.85, 30],
    'Medium':['<DETAILED_CAPTION>',0.5, 60],
    'Long':['<MORE_DETAILED_CAPTION>',0.35, 100]
}
extension = ".txt"

if DataClean == True :
  %cd /content
  clean_directory(ImageFolder)
if Caption_Image != "None":
  caption_dir(ImageFolder,Cap_prompt[Caption_Length])

list_train_dir = []
list_train_dir.append(ImageFolder)

if AddFolderName:
  for dir in list_train_dir:
    add_forder_name(dir)
if Custom_Caption != "":
  for dir in list_train_dir:
    process_dir(dir, Custom_Caption, Append, Remove_Caption)


if ControlFolder != "":
  import shutil
  shutil.copytree(ControlFolder, f"/content/control_folder/{ControlFolder.split('/')[-2]}", dirs_exist_ok=True)
  ControlFolder = f"/content/control_folder/{ControlFolder.split('/')[-2]}"
  config_control_image(ImageFolder,ControlFolder)


In [ ]:
#@title 🛠️ 3. Cấu hình train

# @markdown [![](https://img.shields.io/badge/Adv-Config-blue)](https://github.com/kohya-ss/musubi-tuner/blob/main/docs/advanced_config.md#specify-time-step-range-for-training)

model_type = "z_image_turbo" # @param ["kontext_dev", "qwen_image", "qwen_image_edit", "qwen_image_edit_2509", "qwen_image_edit_2511", "z_image_turbo", "finetuning-qwen_image", "finetuning-qwen_image_edit", "finetuning-qwen_image_edit_2509", "finetuning-qwen_image_edit_2511", "finetuning-z_image_turbo"]
# %run {repo_dir}/Config.ipynb

# @markdown 💡 `Model tuỳ chọn (Nhận link tải civitai, huggingface, drive path)`
model_train = "" # @param {type:'string'}
output_dir = "/content/drive/MyDrive/SD-Data/Lora" # @param {type:'string'}
output_name = "lora_name" # @param {type:'string'}
resolution = "1024" # @param {type:'string'}
batch_size = 1 # @param {type:'number'}
resize_control = False #@param {type:"boolean"}
image_dir = ImageFolder
num_repeats = 20 # @param {type:'number'}
max_train_epochs = 4 # @param {type:'number'}
max_train_steps = 0 # @param {type:'number'}
save_every_n_epochs = 1 # @param {type:'number'}
save_last_n_epochs = 4 # @param {type:'number'}
save_every_n_steps = 0 # @param {type:'number'}

# @markdown *️⃣ `Điều chỉnh Learning Rate`

# @markdown 💡 `Nên đặt lr = 1e-4 với lora, 1e-6 với finetuning`

learning_rate = 1e-4 # @param {type:'number'}
optimizer_type = "adamw8bit" # @param ["adamw", "adamw8bit", "adafactor"]
# @markdown 💡 `Mặc định finetuning sẽ dùng adafactor`
lr_scheduler = "constant" # @param [ "constant", "linear", "cosine", "cosine_with_restarts", "polynomial", "constant_with_warmup", "adafactor","rex"]
lr_poly_power = 1 # @param {type:'number'}
lr_warmup_steps = 10 #@param {type:"number"}
lr_restarts_num_cycles = 4 #@param {type:"number"}
network_dim = 32 # @param {type:'number'}
network_alpha = 16 # @param {type:'number'}

# @markdown 💡 `Phương pháp timestep_sampling: shift - Cân bằng, logsnr - Tổng thể, sigma - Chi tiết`

timestep_sampling = "shift" # @param ["sigma","uniform","sigmoid","shift","flux_shift","qwen_shift","logsnr","qinglong_flux","qinglong_qwen"]
# @markdown 💡 `Điều chỉnh khoảng timestep: Thấp (vd 0-200) - Train chi tiết (face, detail), Cao (400-1000) - Train tổng thể (màu, ánh sáng, phong cách)`
min_timestep = 0 # @param {"type":"slider","min":0,"max":1000,"step":1}
max_timestep = 1000 # @param {"type":"slider","min":0,"max":1000,"step":1}
preserve_distribution_shape = False #@param {type:"boolean"}
# @markdown *️⃣ `Các tuỳ chọn mở rộng`

# @markdown ▪️ `fp8_base, fp8_vl, gradient_checkpointing để giảm VRAM`

# @markdown ▪️ `full_bf16 để giảm VRAM khi finetuning`

# @markdown ▪️ `blocks_to_swap để giảm VRAM khi train Qwen`

fp8_base = True #@param {type:"boolean"}
fp8_vl = True #@param {type:"boolean"}
gradient_checkpointing = True #@param {type:"boolean"}
full_bf16 = True #@param {type:"boolean"}
blocks_to_swap = False #@param {type:"boolean"}

# @markdown *️⃣ `Bật/Tắt các tính năng mở rộng, bao gồm (info, sampler, sao lưu, log)`
extra_option = True #@param {type:"boolean"}
author = 'stablediffusion.vn' # @param {"type":"string"}
description = 'trainlora.vn' # @param {"type":"string"}
sample_prompt = "" #@param {type:'string'}
sample_every_n_steps = 100 # @param {type:'number'}
# @markdown 🟡 `Train kontext, qwen-edit sẽ cần một ảnh đầu để chạy được sample, bỏ qua sẽ chọn tự động`
sample_image_path = "" #@param {type:'string'}
wandb_api_key = '' #@param {type:'string'}
save_state = False #@param {type:"boolean"}
save_state_on_train_end = False #@param {type:"boolean"}
resume = "" #@param {type:'string'}

#default
resolution = [int(x) for x in resolution.split(",")]
if len(resolution) == 1:
  resolution = [resolution[0],resolution[0]]
sample_size = resolution
dataset = os.path.join(root_dir,"dataset.toml")

data_config = fun_data_config()
fun_dataset_file()
config = fun_config() if "finetuning" not in model_type else fun_config_finetuning()
extra = fun_extra()
folder_train = ImageFolder
fun_prompt(sample_prompt, sample_image_path)
cache_latents_config = fun_cache_latents_config()
cache_text_encoder_config = fun_cache_text_encoder_config()

arg = f'accelerate launch {dic2arg(op)} {dic2arg(config)} {dic2arg(extra) if extra_option else ""}'
print(arg)
arg_latents_config = dic2arg(cache_latents_config)
print(arg_latents_config)
arg_text_encoder_config = dic2arg(cache_text_encoder_config)
print(arg_text_encoder_config)
if "z_image" in model_type:
  convert_arg = f"python src/musubi_tuner/networks/convert_z_image_lora_to_comfy.py {output_dir}/{output_name}_{model_type}/{output_name}.safetensors {output_dir}/{output_name}_{model_type}/{output_name}_comfy.safetensors"
  print(convert_arg)
print("Done")

In [ ]:
#@title ⏳ 3. Training
%cd /content/musubi-tuner
run_cache = True #@param {type:"boolean"}
if run_cache:
  !{arg_latents_config}
  !{arg_text_encoder_config}
!{arg}

if "z_image" in model_type:
  !{convert_arg}

AutoDisconect = False # @param {"type":"boolean"}
if AutoDisconect:
  from time import sleep
  sleep(3*60)
  from google.colab import runtime
  runtime.unassign()

# 🎥 III. Train Video (Wan)

In [ ]:
#@title 📂 2. Kết nối - xử lý data

Image_train = True #@param {type:"boolean"}
ImageFolder = "/content/drive/MyDrive/SD-Data/TrainData"  # @param {type:'string'}
Video_train = False #@param {type:"boolean"}
VideoFolder = ""  # @param {type:'string'}
DataClean = False #@param {type:"boolean"}
No_gen_caption = False #@param {type:"boolean"}
# @markdown 🟡 `OCR Prompt`

# @markdown 💡 `Florence yêu cầu khởi động lại phiên (Ctr + M + .) và chạy lại từ đầu`

# @markdown 💡 `Tạo caption tự động cho video chỉ hỗ trợ dùng api qua gemini`
Caption_Image = 'Florence' # @param ['None', 'Florence', 'APIGemini | 2.5 Flash', ' APIGemini | 2.5 Pro', 'APIGemini | 2.5 Flash Lite',  'APIOpenAI | GPT 5', 'APIOpenAI | GPT 5-mini', 'APIOpenAI | GPT 5-nano']
Caption_Video = 'None' # @param ['None', 'APIGemini | 2.5 Flash', ' APIGemini | 2.5 Pro', 'APIGemini | 2.5 Flash Lite']
Caption_Length = "Medium" # @param ["Short","Medium","Long"]
# @markdown 🟡 `API Caption`

# @markdown [![](https://img.shields.io/badge/Gemini-API-blue)](https://aistudio.google.com/app/apikey)
APIkey = "" # @param {type:'string'}
API_Prompt = "" # @param {type:'string'}
# @markdown 🟡 `Thêm caption tuỳ chọn`
Custom_Caption = "" # @param {type:'string'}
AddFolderName = False #@param {type:"boolean"}
Remove_Caption = False #@param {type:"boolean"}
Append = False #@param {type:"boolean"}

if No_gen_caption == True:
  Caption_Image = 'None'
  Caption_Video = 'None'
  Custom_Caption = ''
  AddFolderName = False

Cap_prompt = {
    'Short':['<CAPTION>',0.85, 30],
    'Medium':['<DETAILED_CAPTION>',0.5, 60],
    'Long':['<MORE_DETAILED_CAPTION>',0.35, 100]
}
extension = ".txt"

if DataClean == True :
  %cd /content
  clean_directory(ImageFolder)
if Caption_Image != "None" and Image_train:
  caption_dir(ImageFolder,Cap_prompt[Caption_Length])

if Caption_Video != "None" and Video_train:
  caption_dir(VideoFolder,Cap_prompt[Caption_Length], True)

list_train_dir = []
if Image_train:
  list_train_dir.append(ImageFolder)
if Video_train:
  list_train_dir.append(VideoFolder)
if AddFolderName:
  for dir in list_train_dir:
    add_forder_name(dir)
if Custom_Caption != "":
  for dir in list_train_dir:
    process_dir(dir, Custom_Caption, Append, Remove_Caption)


In [ ]:
#@title 🛠️ 3. Cấu hình train

# @markdown [![](https://img.shields.io/badge/Adv-Config-blue)](https://github.com/kohya-ss/musubi-tuner/blob/main/docs/advanced_config.md#specify-time-step-range-for-training)

model_type = "wan22_i2v_14B" # @param ["wan21_t2v_14B", "wan21_i2v_14B", "wan22_t2v_14B", "wan22_i2v_14B"]
# %run {repo_dir}/Config.ipynb
output_dir = "/content/drive/Shareddrives/StableDiffusion/Lora" # @param {type:'string'}
output_name = "lora_name" # @param {type:'string'}
#value
# @markdown 💡 `Tuỳ chỉnh theo tỉ lệ khung hình và lượng frame. Train hình tương đương frame = 1`
resolution = "720,1080" # @param {type:'string'}
batch_size = 1 # @param {type:'number'}
image_dir = ImageFolder
video_dir = VideoFolder
num_repeats = 20 # @param {type:'number'}
max_train_epochs = 5 # @param {type:'number'}
max_train_steps = 0 # @param {type:'number'}
save_every_n_epochs = 1 # @param {type:'number'}
save_last_n_epochs = 0 # @param {type:'number'}
save_every_n_steps = 0 # @param {type:'number'}
# @markdown 💡 `Phương pháp timestep_sampling: shift - Cân bằng, logsnr - Tổng thể, sigma - Chi tiết`
timestep_sampling = "shift" # @param ["sigma","uniform","sigmoid","shift","flux_shift","qwen_shift","logsnr","qinglong_flux","qinglong_qwen"]
# @markdown 💡 `Điều chỉnh khoảng timestep: Thấp (vd 0-200) - Train chi tiết (face, detail), Cao (400-1000) - Train tổng thể (màu, ánh sáng, phong cách)`
min_timestep = 0 # @param {"type":"slider","min":0,"max":1000,"step":1}
max_timestep = 1000 # @param {"type":"slider","min":0,"max":1000,"step":1}
preserve_distribution_shape = False #@param {type:"boolean"}
# @markdown 💡 `Điều chỉnh khoảng train model low - hight trong wan22 (i2v - 900, t2v - 875)`
timestep_boundary = 875 # @param {"type":"slider","min":0,"max":1000,"step":1}
# @markdown 💡 `Các chế độ xử lý video (bỏ qua khi bỏ chọn video_train)`

# @markdown ▪️ `head - Lấy trực tiếp các frame từ đầu với lượng frame = target_frames`

# @markdown ▪️ `chunk - Chia video thành từng đoạn theo với lượng frame = target_frames`

# @markdown ▪️ `slide - Trích frame = target_frames và cách nhau frame_stride`

# @markdown ▪️ `uniform - Chia đều video thành frame_sample đoạn, mỗi đoạn trích suất frame = target_frames`

# @markdown ▪️ `full - Trích xuất toàn bộ video và giới hạn không quá max_frames`

# @markdown 💡 `target_frames - Lượng frame được cắt | VD:1,25,33 - 3 phương án với 1, 25, 33 frame`

frame_extraction = "chunk" # @param ["head","chunk","slide","uniform","full"]
target_frames = "25" # @param {type:'string'}
frame_stride = 1 # @param {type:'number'}
frame_sample = 1 # @param {type:'number'}
max_frames = 33 # @param {type:'number'}

# @markdown *️⃣ `Điều chỉnh Learning Rate`
learning_rate = 1e-4 # @param {type:'number'}
optimizer_type = "adamw8bit" # @param ["adamw", "adamw8bit", "adafactor"]
lr_scheduler = "constant" # @param [ "constant", "linear", "cosine", "cosine_with_restarts", "polynomial", "constant_with_warmup", "adafactor","rex"]
lr_poly_power = 1 # @param {type:'number'}
lr_warmup_steps = 10 #@param {type:"number"}
lr_restarts_num_cycles = 4 #@param {type:"number"}
network_dim = 32 # @param {type:'number'}
network_alpha = 16 # @param {type:'number'}

# @markdown *️⃣ `Các tuỳ chọn mở rộng`

# @markdown ▪️ `fp8_base, gradient_checkpointing để giảm VRAM`

fp8_base = True #@param {type:"boolean"}
gradient_checkpointing = True #@param {type:"boolean"}

# @markdown *️⃣ `Bật/Tắt các tính năng mở rộng, bao gồm (info, sampler, sao lưu, log)`
extra_option = True #@param {type:"boolean"}
author = 'stablediffusion.vn' # @param {"type":"string"}
description = 'trainlora.vn' # @param {"type":"string"}
sample_prompt = "" #@param {type:'string'}
sample_every_n_steps = 200 # @param {type:'number'}
# @markdown 🟡 `Train model i2v (Image to video) sẽ cần một ảnh frame đầu để chạy được sample`
sample_image_path = "" #@param {type:'string'}
wandb_api_key = '' #@param {type:'string'}
save_state = False #@param {type:"boolean"}
save_state_on_train_end = False #@param {type:"boolean"}
resume = "" #@param {type:'string'}

resolution = [int(x) for x in resolution.split(",")]
if len(resolution) == 1:
  resolution = [resolution[0],resolution[0]]
sample_size = resolution
target_frames = [int(x) for x in target_frames.split(",")]
dataset = os.path.join(root_dir,"dataset.toml")

data_config = fun_data_config()
fun_dataset_file_video()
config = fun_config_video()
extra = fun_extra()
folder_train = ImageFolder if Image_train else VideoFolder
fun_prompt_video(sample_prompt, sample_image_path)
cache_latents_config = fun_cache_latents_config_video()
cache_text_encoder_config = fun_cache_text_encoder_config_video()
if "i2v" in model_type and extra_option and sample_image_path == "":
  print('Không được bỏ trống sample_image_path')
else:
  arg = f'accelerate launch {dic2arg(op)} {dic2arg(config)} {dic2arg(extra) if extra_option else ""}'
  print(arg)
  arg_latents_config = dic2arg(cache_latents_config)
  print(arg_latents_config)
  arg_text_encoder_config = dic2arg(cache_text_encoder_config)
  print(arg_text_encoder_config)
  print("Done")

In [ ]:
#@title ⏳ 3. Training
%cd /content/musubi-tuner
run_cache = True #@param {type:"boolean"}
if run_cache:
  !{arg_latents_config}
  !{arg_text_encoder_config}
!{arg}

AutoDisconect = False # @param {"type":"boolean"}
if AutoDisconect:
  from time import sleep
  sleep(3*60)
  from google.colab import runtime
  runtime.unassign()